In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [2]:
df_workout=pd.read_csv("DL_Workout.csv")

In [3]:
df_workout.head()

,Age,Gender,Weight (kg),Height (m),BMI,BMI_Category,Experience_Level,Session_Duration (hours),Calories_Burned,Name of Exercise,...,target_muscles,focus_area,strength,cardio,hiit,yoga,calories_norm,difficulty_norm,weight_gain_score,weight_loss_score
0,34.91,1,65.27,1.62,24.87,1,14,1.00,1080.90,Decline Push-ups,...,"['lower_chest', 'triceps']",upperbody,1,0,0,0,0.845312,0.0,0.476797,0.20
1,23.37,0,56.41,1.55,23.48,1,14,1.37,1809.91,Bear Crawls,...,"['shoulders', 'core', 'legs']",fullbody,0,0,1,0,0.902949,1.0,0.385442,0.50
2,33.20,0,58.98,1.67,21.15,1,2,0.91,802.26,Dips,...,"['chest', 'triceps']",upperbody,1,0,0,0,0.912713,1.0,0.486907,0.35
3,38.69,0,93.78,1.70,32.45,3,12,1.10,1450.79,Mountain Climbers,...,"['core', 'shoulders', 'legs']",fullbody,0,1,1,0,0.881167,0.0,0.582175,0.60
4,45.09,1,52.42,1.88,14.83,0,13,1.08,1166.40,Bicep Curls,...,['biceps'],upperbody,1,0,0,0,0.793050,0.0,0.468958,0.20


In [ ]:
feature_cols = [
    "Age",
    "Gender",
    "BMI",
    "Experience_Level",
    "Session_Duration (hours)",
    "Difficulty Level",
    "Reps",
    "strength",
    "cardio",
    "hiit",
    "yoga",
    "weight_loss_score"
]

X = df_workout[feature_cols].values
y = df_workout["weight_loss_score"].values  # or weight_gain_score


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# normalise the input features

In [5]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


# convert data to pytorch tensors 
## pytorch works with tensors and not Numpyarrays

In [6]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_t  = torch.tensor(X_test, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [7]:
class WorkoutRecommender_loss(nn.Module):
    def __init__(self, input_dim):
        super(WorkoutRecommender_loss, self).__init__()

        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 1)
    #fc1 ,fc2=hidden layers
        
        self.relu = nn.ReLU() #for learning non linear patterns
        self.sigmoid = nn.Sigmoid() #for output between 0 and 1

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.out(x))
        return x


In [8]:
model = WorkoutRecommender_loss(input_dim=X_train.shape[1])

criterion = nn.MSELoss()        # regression loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
epochs = 60

for epoch in range(epochs):

    model.train()

    optimizer.zero_grad()              # clear old gradients
    outputs = model(X_train_t)         # forward pass
    loss = criterion(outputs, y_train_t)  # calculate error
    loss.backward()                    # backpropagation
    optimizer.step()                   # update weights

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [5/60], Loss: 0.0787
Epoch [10/60], Loss: 0.0718
Epoch [15/60], Loss: 0.0657
Epoch [20/60], Loss: 0.0605
Epoch [25/60], Loss: 0.0558
Epoch [30/60], Loss: 0.0512
Epoch [35/60], Loss: 0.0464
Epoch [40/60], Loss: 0.0411
Epoch [45/60], Loss: 0.0355
Epoch [50/60], Loss: 0.0298
Epoch [55/60], Loss: 0.0248
Epoch [60/60], Loss: 0.0208


In [10]:
model.eval()

with torch.no_grad():
    test_preds = model(X_test_t)
    test_loss = criterion(test_preds, y_test_t)

print("Test Loss:", test_loss.item())


Test Loss: 0.021245308220386505


In [11]:

user_input = {
    "age": 30,
    "weight": 70,
    "height": 1.65,
    "gender": 1   # male
}



In [16]:
def recommend_workouts(user_input, exercise_df, model, scaler, top_n=5):

    bmi = user_input["weight"] / (user_input["height"] ** 2)
    model.eval()
    recommendations = []

    for _, row in exercise_df.iterrows():

        X = [
            user_input["age"],
            user_input["gender"],
            bmi,
            row["Experience_Level"],
            row["Session_Duration (hours)"],
            row["Difficulty Level"],
            row["Reps"],
            row["strength"],
            row["cardio"],
            row["hiit"],
            row["yoga"]
        ]

        X = scaler.transform([X])
        X_t = torch.tensor(X, dtype=torch.float32)

        with torch.no_grad():
            score = model(X_t).item()
        col=["Name of Exercise","Difficulty Level","target_muscles","focus_area"]
        recommendations.append((row[col], score))

        recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]


In [17]:

top_exercises = recommend_workouts(
    user_input,
    df_workout,
    model,
    scaler,
    top_n=5
)


In [18]:
top_exercises

[(Name of Exercise             Frog Jumps
  Difficulty Level                      0
  target_muscles      ['glutes', 'quads']
  focus_area                    lowerbody
  Name: 6411, dtype: object,
  0.5161949992179871),
 (Name of Exercise             Frog Jumps
  Difficulty Level                      0
  target_muscles      ['glutes', 'quads']
  focus_area                    lowerbody
  Name: 17492, dtype: object,
  0.5136634111404419),
 (Name of Exercise                Mountain Climbers
  Difficulty Level                                0
  target_muscles      ['core', 'shoulders', 'legs']
  focus_area                               fullbody
  Name: 12797, dtype: object,
  0.5135391354560852),
 (Name of Exercise    Jumping Jacks
  Difficulty Level                0
  target_muscles      ['full_body']
  focus_area               fullbody
  Name: 5018, dtype: object,
  0.5126786231994629),
 (Name of Exercise             Frog Jumps
  Difficulty Level                      0
  target_muscles  

In [19]:
checkpoint = {
    "model_state_dict": model.state_dict(),
    "input_dim": X.shape[1],
    "feature_cols": feature_cols,          # VERY IMPORTANT
    "scaler": scaler,                       # normalization
}
torch.save(checkpoint, "workout_recommender(loss).pth")


In [20]:
# for weight gain

In [22]:
feature_cols = [
    "Age",
    "Gender",
    "BMI",
    "Experience_Level",
    "Session_Duration (hours)",
    "Difficulty Level",
    "Reps",
    "strength",
    "cardio",
    "hiit",
    "yoga"
]

X1 = df_workout[feature_cols].values
y1 = df_workout["weight_gain_score"].values  # or weight_gain_score


In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X1, y1, test_size=0.2, random_state=42
)


In [24]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [25]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_t  = torch.tensor(X_test, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [26]:
class WorkoutRecommender_gain(nn.Module):
    def __init__(self, input_dim):
        super(WorkoutRecommender_gain, self).__init__()

        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 1)
    #fc1 ,fc2=hidden layers
        
        self.relu = nn.ReLU() #for learning non linear patterns
        self.sigmoid = nn.Sigmoid() #for output between 0 and 1

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.out(x))
        return x

In [27]:
model_gain = WorkoutRecommender_gain(input_dim=X_train.shape[1])

criterion = nn.MSELoss()
optimizer = optim.Adam(model_gain.parameters(), lr=0.001)

epochs = 60
for epoch in range(epochs):
    optimizer.zero_grad()
    preds = model_gain(X_train_t)
    loss = criterion(preds, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")



Epoch 5, Loss: 0.0147
Epoch 10, Loss: 0.0136
Epoch 15, Loss: 0.0130
Epoch 20, Loss: 0.0126
Epoch 25, Loss: 0.0120
Epoch 30, Loss: 0.0114
Epoch 35, Loss: 0.0109
Epoch 40, Loss: 0.0103
Epoch 45, Loss: 0.0096
Epoch 50, Loss: 0.0089
Epoch 55, Loss: 0.0082
Epoch 60, Loss: 0.0074


In [28]:
model_gain.eval()
with torch.no_grad():
    test_preds = model_gain(X_test_t)
    test_loss = criterion(test_preds, y_test_t)

print("Weight Gain Model Test Loss:", test_loss.item())


Weight Gain Model Test Loss: 0.007734752260148525


In [29]:
checkpoint = {
    "model_state_dict": model.state_dict(),
    "input_dim": X.shape[1],
    "feature_cols": feature_cols,          # VERY IMPORTANT
    "scaler": scaler,                       # normalization
}
torch.save(checkpoint, "workout_recommender(gain).pth")
